### 1. Saisie des données à l’aide du module sqlite3


__Test de code__

In [5]:
import sqlite3
try:
    with sqlite3.connect('C:\\temp\\io\\database\\insurancedb.db') as conn:
        curseur = conn.cursor()
        curseur.execute('SELECT * FROM insurance ORDER BY policyID DESC;')
        print(curseur.fetchall())
        conn.close()
except sqlite3.Error as error:
    print(error)
            

[(995932, 'FL', 'CLAY COUNTY'), (785275, 'FL', 'CLAY COUNTY'), (448094, 'FL', 'CLAY COUNTY'), (433512, 'FL', 'CLAY COUNTY'), (333743, 'FL', 'CLAY COUNTY'), (223488, 'FL', 'CLAY COUNTY'), (206893, 'FL', 'CLAY COUNTY'), (172534, 'FL', 'CLAY COUNTY'), (119736, 'FL', 'CLAY COUNTY')]
Cannot operate on a closed database.


In [7]:
import sqlite3

class Personne:
    def __init__(self,nom,age):
        self.nom = nom 
        self.age = age
    
    def __conform__(self,protocol):
        if protocol is sqlite3.PrepareProtocol:
            return f'Le nom est {self.nom} et l\'age est {self.age}'

conn = sqlite3.connect(':memory:')
curseur = conn.cursor()

moi = Personne('Béchir Béjaoui', 40)
curseur.execute('SELECT ?',(moi,))
print(curseur.fetchone()[0])

Le nom est Béchir Béjaoui et l'age est 40


### 2.Saisie des données à l’aide du module pyodbc
 

In [24]:
import pyodbc
try:
    with pyodbc.connect('DRIVER={SQLite3 ODBC Driver};'
                        'SERVER=localhost;'
                        'DATABASE=C:\\temp\\io\\database\\insurancedb.db;'
                        'Trusted_connection=yes;'
                        ) as conn:
        curseur = conn.cursor()
        curseur.execute('SELECT * FROM insurance ORDER BY policyID DESC;')
        
        for row in curseur:
            print(row)
        
        conn.close()
except pyodbc.Error as error:
    print(error)

(995932, 'FL', 'CLAY COUNTY')
(785275, 'FL', 'CLAY COUNTY')
(448094, 'FL', 'CLAY COUNTY')
(433512, 'FL', 'CLAY COUNTY')
(333743, 'FL', 'CLAY COUNTY')
(223488, 'FL', 'CLAY COUNTY')
(206893, 'FL', 'CLAY COUNTY')
(172534, 'FL', 'CLAY COUNTY')
(119736, 'FL', 'CLAY COUNTY')
('HY000', 'The driver did not supply an error!')


### 3. Saisie des données à l’aide du module cx_oracle


In [25]:
!type c:\temp\io\organisme.sql

/*CREATE OR REPLACE PACKAGE BODY organisme AS */

/*DROP TABLE Employee*/
CREATE TABLE Employee(
     employee_id  NUMBER NOT NULL,
	 employee_name  VARCHAR2(20) NOT NULL,
	 superior_id  NUMBER
);

/*DROP TABLE Departement*/
CREATE TABLE Departement(
    department_id  NUMBER NOT NULL,
    department_name  VARCHAR2(20) NOT NULL,
    manager_id NUMBER
);

ALTER TABLE Employee ADD CONSTRAINT employee_pk PRIMARY KEY(employee_id);
ALTER TABLE Departement ADD CONSTRAINT departement_pk PRIMARY KEY(department_id);
ALTER TABLE Departement ADD CONSTRAINT employee_fk FOREIGN KEY(manager_id)
REFERENCES Employee(employee_id);


INSERT INTO Employee(employee_id,employee_name,superior_id) VALUES(1,'Jon Doe',NULL);
INSERT INTO Employee(employee_id,employee_name,superior_id) VALUES(2,'Maria Cleber',1);
INSERT INTO Employee(employee_id,employee_name,superior_id) VALUES(3,'Marin Marshall',1);
INSERT INTO Employee(employee_id,employee_name,superior_id) VALUES(4,'Peter Tank',2);
INSERT INTO Employee(emplo

In [30]:
import cx_Oracle

class Organisme:
    def __init__(self):
        self.__conn=cx_Oracle.Connection('bechir/pa$$w0rd@//localhost:1521/XE')
        self.__cursor = self.__conn.cursor()
        
    def __exit__(self,typ,valeur,trace):
        self.__cursor.close()
        self.__conn.close()
    
    def add_departement(self,p_departement_id,p_departement_name,p_manager_ID):
        self.__cursor.callproc('add_departement',[p_departement_id,p_departement_name,p_manager_ID])
        return 0

    def add_employee(self,p_employee_id,p_employee_name,p_superior_ID):
        self.__cursor.callproc('add_employee',[p_employee_id,p_employee_name,p_superior_ID])
        return 0
    
    def get_employees(self):
        l_cursor = self.__cursor.var(cx_Oracle.CURSOR)
        l_emp = self.__cursor.callproc('get_employees',[l_cursor])
        return l_emp
    
    def get_departments(self):
        l_cursor = self.__cursor.var(cx_Oracle.CURSOR)
        l_dep = self.__cursor.callproc('get_departements',[l_cursor])
        return l_dep

__Test  de la classe Organisme__

In [31]:
organisme = Organisme()
employees = organisme.get_employees()
employees = (employees)[0].fetchall()
print(employees)

[(1, 'Jon Doe', None), (2, 'Maria Cleber', 1), (3, 'Marin Marshall', 1), (4, 'Peter Tank', 2), (5, 'Dalia Mtir', 2), (6, 'Susanne Lubser', 2), (7, 'Mike Glims', 2), (8, 'Maharaj Mahatra', 3), (9, 'Ferjani Sassi', 3), (10, 'Samara Dust', 2), (11, 'Ismail Kano', 3), (12, 'Mustapha Khir Allah', 2), (13, 'Samia Mahari', 3), (14, 'Fatma Mortadha', 3), (15, 'Himo Hazkle', 3), (16, 'Michael Martin', 2)]


In [33]:
organisme = Organisme()
departements = organisme.get_departments()
departements = (departements)[0].fetchall()
print(departements)

[(1, 'Administration', 2), (2, 'RD', 3)]


### 4. Saisie des données à l’aide de  l’orm sqlalchemy


In [38]:
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('oracle://bechir:pa$$w0rd@XE')
engine.connect()
print('Connecté ...')
Base =declarative_base()
Session = sessionmaker(bind = engine)


class Employee(Base):
    __tablename__='employee'
    identifiant = Column('employee_id',Integer,primary_key=True)
    nom =Column('employee_name',String)
    superieur = Column('superior_id',String)
    
    def __init__(self,identifiant,nom,superieur):
        self.identifiant = identifiant
        self.nom = nom 
        self.superieur =superieur
        
class Departement(Base):
    __tablename__='departement'
    identifiant = Column('department_id',Integer,primary_key=True)
    nom =Column('department_name',String)
    manager_id = Column('manager_id',Integer,foreign_key=True)
    
    def __init__(self,identifiant,nom,manager_id):
        self.identifiant = identifiant
        self.nom = nom 
        self.manager_id =manager_id

session = Session()

employees = session.query(Employee).all()
print('Les employées:')
for row in employees:
    print(f'{row.identifiant}  {row.nom} {row.superieur}')

departements = session.query(Departement).all()
print('Les dépratements:')
for row in departements:
    print(f'{row.identifiant}  {row.nom} {row.manager_id}')

Connecté ...
Les employées:
1  Jon Doe None
2  Maria Cleber 1
3  Marin Marshall 1
4  Peter Tank 2
5  Dalia Mtir 2
6  Susanne Lubser 2
7  Mike Glims 2
8  Maharaj Mahatra 3
9  Ferjani Sassi 3
10  Samara Dust 2
11  Ismail Kano 3
12  Mustapha Khir Allah 2
13  Samia Mahari 3
14  Fatma Mortadha 3
15  Himo Hazkle 3
16  Michael Martin 2
Les dépratements:
1  Administration 2
2  RD 3


### 5. Saisie des données à l’aide du module pymongo


In [1]:
import pymongo

class Insurances:
    def __init__(self,db,collection):
        self.__client = pymongo.MongoClient('mongodb://localhost:27017/')
        self.db = db
        self.collection = collection
        self.__db = self.__client[db]
        self.__collection = self.__db[collection]
   
    def __exit__(self,typ,valeur,trace):
        self.__client.close()
    
    def get_doc(self):
        liste = []
        for element in self.__collection.find():
            liste.append(element)
        return liste

    
        

__Test de code__

In [2]:
insurance = Insurances('insurance','docs')
resultat = insurance.get_doc()
for element in resultat:
    print(element)


{'_id': ObjectId('5c9e11a7eb4982ed18ae0313'), 'record1': {'policyID': 119736, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record2': {'policyID': 448094, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record3': {'policyID': 206893, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record4': {'policyID': 333743, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record5': {'policyID': 172534, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record6': {'policyID': 785275, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record7': {'policyID': 995932, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record8': {'policyID': 223488, 'statecode': 'FL', 'county': 'CLAY COUNTY'}, 'record9': {'policyID': 433512, 'statecode': 'FL', 'county': 'CLAY COUNTY'}}
